# Generate Controls

> LTGenius (LTG) is an AI-driven engine that plays a pivotal role in the governance and compliance ecosystem by automating the generation of policies and controls and performing compliance gap analysis. Built as a byproduct of the LockThreat platform, LTGenius optimizes and streamlines policy and control generation and identifies gaps in policies and controls. 

In [ ]:
#| default_exp policy/generate_controls

In [ ]:
#| export
from app.policy.helper.llm_functions import get_openai_embedding, init_azure_llm
from app.policy.helper.tools import apply_markdown, apply_markdown_gap, merge_sets, create_policy_markdown
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
from app.policy.helper.prompts import policy_prompt_template, controls_prompt_template, policy_gap_prompt_template, policy_gap_output

In [ ]:
#| export
##
## PROCESSOR ##
##
def combine_similar_controls(
    standards, # List of standards
    threshold=0.7 # Matched threshold
)->list: # List of merged controls 
    # Apply markdown to all standards
    # standards = load_standards()
    for idx, standard in enumerate(standards):
        for domain in standard['Controls']:
            domain['Markdown'] = apply_markdown(domain, idx)
    arrays = [standard['Controls'] for standard in standards]
    unique_items = []
    array_indices = []

    for array_index, sublist in enumerate(arrays):
        unique_items.extend(sublist)
        array_indices.extend([array_index] * len(sublist))
    ids = [item['DomainId'] for item in unique_items]  # Define 'ids' here
    # ids = [f"{array_indices[index]}:{item['DomainId']}" for index, item in enumerate(unique_items)]  # Define 'ids' here


    # Extract all controls and their associated domain IDs
    all_controls = []
    control_domain_ids = [] 
    for item in unique_items:
        for control in item['Controls']:
            all_controls.append(control['markdown'])
            control_domain_ids.append(control['controlId'])
    # print(all_controls)
    # Get embeddings for each control
    embeddings = np.array(get_openai_embedding(all_controls))

    # Compute cosine distances
    cosine_dist = cosine_distances(embeddings)

    # print("cosine_dist=",cosine_dist)
    # Create a mapping from control index to its domain ID
    cosine_dist2 = cosine_dist

    ### Applying triangular indices to reduce cost
    cosine_dist2 = np.triu(cosine_dist,)
    # Get the lower triangular indices (excluding the diagonal)
    lower_triangular_indices = np.tril_indices_from(cosine_dist2, k=-1)

    # Replace elements at these indices with '1'
    cosine_dist2[lower_triangular_indices] = '1'

    #make diagonal 1 of cosine_dist2 matrix
    np.fill_diagonal(cosine_dist2, 1)
    cosine_dist = cosine_dist2

    control_to_domain = {i: control_domain_ids[i] for i in range(len(all_controls))}
    
    # Identify pairs of controls with cosine similarity above the threshold
    similar_pairs = []

    # for i, j in product(range(len(cosine_dist)), range(len(cosine_dist))):
    already_matched1 = []
    already_matched2 = []
    for i in range(len(cosine_dist)):
        icnt = 0
        for j in range(len(cosine_dist)):
            if i != j and cosine_dist[i][j] <= 1 - threshold and j not in already_matched2: #and i not in already_matched1: 
                domain1 = control_to_domain[i]
                domain2 = control_to_domain[j]
                i_d_c1 =  domain1.split(":")
                i_d_c2 =  domain2.split(":")
                
                if i_d_c1[0] != i_d_c2[0]:
                    if icnt >=int(i_d_c2[0]): 
                        continue
                    similar_pairs.append((domain1, domain2))
                    already_matched1.append(i)
                    already_matched2.append(j)
                    icnt = int(i_d_c2[0])

    # print("similar_pairs=",similar_pairs)
    # Create a graph where nodes are domain IDs and edges represent similarity
    graph = {domain_id: set() for domain_id in ids}
    for domain1, domain2 in similar_pairs:
        d1 = domain1.split(":")
        d1 = d1[0] + ':' + d1[1]
        d2 = domain2.split(":")
        d2 = d2[0] + ':' + d2[1]
        graph[d1].add(d2)
        graph[d2].add(d1)
    # Find connected components in the graph
    visited = set()
    clusters = []
    for domain_id in ids:
        if domain_id not in visited:
            cluster = set()
            stack = [domain_id]
            while stack:
                current_domain = stack.pop()
                if current_domain not in visited:
                    visited.add(current_domain)
                    cluster.add(current_domain)
                    stack.extend(graph[current_domain])
            clusters.append(cluster)
    # print("clusters=",clusters)

    # Merge domains and controls within each cluster
    merged_list = []
    merged_set = []
    for cluster in clusters:

        merged_item = {
            'id': '-'.join(map(str, cluster)),
            'domain': '-'.join(item['Domain'] for item in unique_items if item['DomainId'] in cluster),
            'Controls': []
        }

        # Gather and merge controls, considering cosine similarity
        seen_control_names = []
        merged_controls = [] # To track merged control IDs
        for d1, d2 in similar_pairs:
            domain1 =  d1[:d1.rfind(":")]
            domain2 =  d2[:d2.rfind(":")]
            if domain1 in cluster and domain2 in cluster:
                control1=next((c for item in unique_items if item['DomainId'] == domain1 for c in item['Controls'] if c['controlId'] == d1), {})
                control2=next((c for item in unique_items if item['DomainId'] == domain2 for c in item['Controls'] if c['controlId'] == d2), {})
                if control1 and control2:
                    control_names_set = set({control1['controlId'], control2['controlId']})
                    control_markdown_set = set({control1['markdown'], control2['markdown']})
                    if control_names_set not in seen_control_names:
                        merged_item['Controls'].append({
                            'id': f"{control1['controlId']}-{control2['controlId']}",
                            # 'name': '-'.join(sorted(control_markdown_set)),
                            'name': control1['name'] +'-'+control2['name'],
                            'desc': [control1['markdown'], control2['markdown']]
                        })

                        if(len(seen_control_names)>0):
                            # seen_control_names=merge_sets(control_names_set,seen_control_names)
                            seen_control_names.append(control_names_set)
                        else:
                            seen_control_names.append(control_names_set)
                        merged_controls.append(control1['controlId'])
                        merged_controls.append(control2['controlId'])
        merged_set.append(seen_control_names)
        # Add unmerged controls
        for domain_id in cluster:
            for control in next((item for item in unique_items if item['DomainId'] == domain_id), {})['Controls']:
                if control['controlId'] not in merged_controls: # Check if control was already merged
                    merged_item['Controls'].append({
                        'id': control['controlId'],
                        'name': control['name'],
                        'description': control['description']
                    })
        merged_list.append(merged_item)    
    return merged_list


In [ ]:
#| export
# Function to extract headings using LangChain
def merge_controls_using_LLM(
    llm, # LLM object 
    text # Control Markdown
)->str: # Merged Description of controls
    "Merge control's description through LLM"
    prompt = controls_prompt_template.format(current_input=text)
    response = llm.invoke(prompt)
    return response.content

In [ ]:
#| export
def merge_controls(
    llm, # LLM object
    merged_list # List of similar controls
)->list: # List of merged controls with updated descriptions
    # for each merged_list
    # find the controls that have '-' in their id and merge them
    for merged_item in merged_list:
        for control in merged_item['Controls']:
            if '-' in control['id']:
                control['description'] = merge_controls_using_LLM(llm, control['desc'])
                del control['desc']
    return merged_list

In [ ]:
#| export
def standard_processor(
    standards # List of standards
):
    print('control api process started')
    llm = init_azure_llm()
    merged_list = combine_similar_controls(standards)
    merged_controls = merge_controls(llm, merged_list)
    print('control api process completed')
    return merged_controls

In [ ]:
standard1 = {
    "StandardName": "ISO/IEC 27001:2022",
    "Controls": [
        {
            "DomainId": "4",
            "Domain": "Context of the organization",
            "Controls": [
                {
                    "controlId": "4.1",
                    "policy": "Context of the organization",
                    "name": "Understanding the organization and its context",
                    "description": "The organization shall determine external and internal issues that are relevant to its purpose and that affect its ability to achieve the intended outcome(s) of its information security management system.",
                },
                {
                    "controlId": "4.2",
                    "policy": "Context of the organization",
                    "name": "Understanding the needs and expectations of interested parties",
                    "description": "The organization shall determine: a) interested parties that are relevant to the information security management system; b) the relevant requirements of these interested parties; c) which of these requirements will be addressed through the information security management system.",
                },
            ],
        }
    ],
}
standard2 = {
    "StandardName": "ISO 9001:2015",
    "Controls": [
        {
            "DomainId": "4",
            "Domain": "Context of the organization",
            "Controls": [
                {
                    "controlId": "4.1",
                    "policy": "Context of the organization",
                    "name": "Understanding the organization and its context",
                    "description": "The organization shall determine external and internal issues that are relevant to its purpose and its strategic direction and that affect its ability to achieve the intended result(s) of its quality management system. \nThe organization shall monitor and review information about these external and internal issues. \nNOTE 1 Issues can include positive and negative factors or conditions for consideration.\nNOTE 2 Understanding the external context can be facilitated by considering issues arising from legal, technological, competitive, market, cultural, social and economic environments, whether international, national, regional or local. \nNOTE 3 Understanding the internal context can be facilitated by considering issues related to values, culture, knowledge and performance of the organization.",
                },
                {
                    "controlId": "4.2",
                    "policy": "Context of the organization",
                    "name": "Understanding the needs and expectations of interested parties",
                    "description": "Due to their effect or potential effect on the organization’s ability to consistently provide products and\nservices that meet customer and applicable statutory and regulatory requirements, the organization\nshall determine:\na) the interested parties that are relevant to the quality management system;\nb) the requirements of these interested parties that are relevant to the quality management system.\nThe organization shall monitor and review information about these interested parties and their\nrelevant requirements.",
                },
            ],
        }
    ],
}
standard_processor([standard1,standard2])

control api process started
control api process completed


[{'id': '0:4-1:4',
  'domain': 'Context of the organization-Context of the organization',
  'Controls': [{'id': '0:4:4.1-1:4:4.1',
    'name': 'Understanding the organization and its context-Understanding the organization and its context',
    'description': 'The organization shall determine external and internal issues that are relevant to its purpose, strategic direction, and that affect its ability to achieve the intended outcomes of its information security and quality management systems. The organization shall monitor and review information about these external and internal issues. Issues can include positive and negative factors or conditions for consideration. Understanding the external context can be facilitated by considering issues arising from legal, technological, competitive, market, cultural, social, and economic environments, whether international, national, regional, or local. Understanding the internal context can be facilitated by considering issues related to values,

In [1]:
#| hide
import nbdev; nbdev.nbdev_export()